In [1]:
import numpy as np
import cv2
from sklearn.utils import shuffle
#import glob
import re
import pims

In [2]:
path = "/home/baug8178/data/DHF1K"
video_path = "/home/baug8178/data/DHF1K/video/{:03d}.AVI"
annotation_path = "/home/baug8178/data/DHF1K/annotation/{:04d}/maps/{:04d}.png"
video_train_frames = "/home/baug8178/data/DHF1K/video_frames/*.jpg"
video_vali_frames = "/home/baug8178/data/DHF1K/video_frames/vali/*.jpg"
video_test_frmaes = "/home/baug8178/data/DHF1K/video_frames/test/*.jpg"

video_train_path = "/home/baug8178/data/DHF1K/video_frames/{}_{}.jpg"
video_vali_path = "/home/baug8178/data/DHF1K/video_frames/vali/{}_{}.jpg"
#video_test_path = "/home/baug8178/data/DHF1K/video_frames/test/{}_{}.jpg"
dhf1k_filename = ".AVI"
train_num_files = 420
val_num_files = 10
batch_size = 8
total_frames =  389592

In [3]:
frames = []
framesobj = []

for file_no in range(1, train_num_files + 1):
  video_file = video_path.format(file_no)
  cap = cv2.VideoCapture(video_file)
  length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  #print( length )

  #framesobj.append (v)
  [frames.append([file_no,i])  for i in range(length)]

In [4]:
total_frames = len(frames)

In [5]:
from sklearn.utils import shuffle

frames = shuffle(frames)

In [6]:
def myGenerator(batch_size):
    
    #number_of_batches  = (train_num_files - 1) / batch_size
    counter = 0   
    #read_size = batch_size
    
    while(1):
        
        read_size = min(batch_size, total_frames - counter)
        train_x = np.empty((read_size, 360,640,3), dtype = np.int8)
        train_y = np.empty((read_size, 360,640,1), dtype = np.int8)
        
        for i in range(read_size):
        
          video_file = video_path.format(frames[i][0])
          v = pims.Video(video_file)
          img = v[frames[i][1]]
          train_x[i,...] = img[None]
         
          anno_train_file = annotation_path.format(int(frames[i][0]), int(frames[i][1]))
          img = cv2.imread(anno_train_file,0)
          img = img.reshape(img.shape[0],img.shape[1], 1)
          train_y[i,...] = img[None]
        
        counter = counter + read_size
        if counter >= total_frames:
           counter = 0

        yield train_x, train_y

In [7]:
class data_processing:
    def __init__(self):
        self.buffer_file = None
        self.buffer_file_y = None
        self.buffer_rem = 0
        self.start_file_num = 1
        return
    
    def read_videos(self,start_file_no , num_files):
        frame_count = []
        frames = []
        for i in range(start_file_no , start_file_no + num_files):
           count = 0
           filename = video_path.format(i) 
           vidcap = cv2.VideoCapture(filename)
           #print(filename)
           success,image = vidcap.read()
           while success:
             frames.append(image)
             success,image = vidcap.read()
             count += 1
        
           frame_count.append(count)
            

        return np.array(frames), np.array(frame_count)
    
    def read_annotations(self,start_file_no , num_files,frame_count):
        frames =[]
        count_i = 0
        for i in range(start_file_no , start_file_no + num_files):
            #folder_path = "annotation\\"+"{:04}".format(i)+"\\maps\\"
            
            for j in range(1,frame_count[count_i]+ 1):
                #print(j)
                filename = annotation_path.format(i,j)
                #print(filename)
                frame = cv2.imread(filename, 0)
                #print(frame.shape)
                frames.append(frame)
            count_i = count_i + 1
         
        frames = np.array(frames)
        #print(frames.shape)
        frames = frames.reshape((frames.shape[0], frames.shape[1], frames.shape[2],1))
        return frames

In [8]:
dp = data_processing()
val_x, val_frame_count = dp.read_videos(421, val_num_files)
val_y = dp.read_annotations(421, val_num_files, val_frame_count)

In [9]:
from sklearn.utils import shuffle

        
print(val_x.shape) 
#print(len(train_frame_count))
print(val_y.shape)


val_x, val_y = shuffle(val_x, val_y, n_samples  = 2000)

(5300, 360, 640, 3)
(5300, 360, 640, 1)


In [10]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
import keras
#import KerasD

/home/baug8178/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
from keras.layers.convolutional import Deconvolution2D,Convolution2D
from keras.layers import UpSampling2D,Activation, Input, ZeroPadding2D
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras.applications.vgg16 import VGG16

def norm_weights(n):
    r = n / 2.0
    xs = np.linspace(-r, r, n)
    x, y = np.meshgrid(xs, xs)
    w = np.exp(-0.5*(x**2 + y**2))
    w /= w.sum()
    return w

def deconv(nb_filter, size, name):
    upsample = UpSampling2D(size=(size, size))
    s = 2 * size + 1
    #w = norm_weights(s)[ :, :,np.newaxis, np.newaxis]
    conv = Convolution2D(
        nb_filter, (s, s),
        name=name,
        activation='linear',
        bias=False,
        border_mode='same')
    return lambda x: conv(upsample(x))

def get_model():

    input_tensor = Input(shape=( 360, 640, 3)) 
    #base_model = VGG16(weights='imagenet', include_top=False ,input_shape=(3, 360, 640))
    model_vgg16_conv = VGG16(weights='imagenet', include_top=False ,input_tensor=input_tensor)

    x = model_vgg16_conv.get_layer('block3_pool').output
    #x = Conv2D(512, (5, 5), init='normal', activation='relu', border_mode='same', name='conv4')(x)
    #x = Conv2D(512, (5, 5), init='normal', activation='relu', border_mode='same', name='conv5')(x)
    #x = Conv2D(256, (7, 7), init='normal', activation='relu', border_mode='same', name='conv6')(x)
    x = Convolution2D(128, (11, 11), init='normal', activation='relu', border_mode='same', name='conv7')(x)
    x = Convolution2D(32 , (11, 11), init='normal', activation='relu', border_mode='same', name='conv8')(x)
    x = Convolution2D(1 , (13, 13), init='normal', activation='relu', border_mode='same', name='conv9')(x)
    x = deconv(1,8, 'deconv')(x)
    output = Activation('sigmoid')(x)

    model = Model(input=input_tensor, output=output )
    model.summary()

    for layer in model_vgg16_conv.layers:
        layer.trainable = False
        
    adam = optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=0.0001, decay=0.0001, amsgrad=False)
    model.compile(optimizer=adam, loss='mae')
    return model

In [12]:
sgf1model = get_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 360, 640, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 360, 640, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 360, 640, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 180, 320, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 180, 320, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 180, 320, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 90, 160, 128)      0         
__________

/home/baug8178/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (11, 11), activation="relu", name="conv7", padding="same", kernel_initializer="normal")`
/home/baug8178/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (11, 11), activation="relu", name="conv8", padding="same", kernel_initializer="normal")`
/home/baug8178/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1, (13, 13), activation="relu", name="conv9", padding="same", kernel_initializer="normal")`
/home/baug8178/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1, (17, 17), name="deconv", activation="linear", padding="same", use_bias=False)`
/home/baug8178/anaconda3/lib/python3.6/site-packages/ip

In [ ]:
import numpy as np
import keras

class MyCbk(keras.callbacks.Callback):

    def __init__(self, model, header):
        self.model_to_save = model.layers[0]
        self.header_name = header

    def on_epoch_end(self, epoch, logs=None):
        self.model_to_save.save(self.header_name + 'model_at_epoch_2_%d.h5' % epoch)
        
cbk = MyCbk(sgf1model,'sgf1model_')
batch_size = 4
sgf1model.fit_generator(myGenerator(batch_size), steps_per_epoch=(total_frames) / batch_size, epochs=20,
                        verbose=1, callbacks=[cbk], validation_data=(val_x, val_y),
                         shuffle=True)

Epoch 1/20
 1613/97533 [..............................] - ETA: 22:23:33 - loss: 2.9109